In [1]:
fjcmakedir="/Users/ploskon/devel/fjcmake/"

In [2]:
import sys

In [3]:
sys.path.append("{}/test/build/python/build/lib/CMakeSwig/fastjet".format(fjcmakedir))
sys.path.append("{}/test/build/python/build/lib/CMakeSwig/recursivetools".format(fjcmakedir))
sys.path.append("{}/test/pythia8235inst/lib".format(fjcmakedir))
sys.path.append("{}/test/build/python/build/lib/CMakeSwig/pythiafjtools".format(fjcmakedir))

In [4]:
import pyfastjet as fj
import pyrecursivetools as rt
import pythia8
import pypythiafjtools as pyfj

In [5]:
def print_jets(jets):
    print("{0:>5s} {1:>10s} {2:>10s} {3:>10s} {4:>12s}".format(
        "jet #", "pt", "rap", "phi", "N particles"))

    for ijet in range(len(jets)):
        jet = jets[ijet]
        constituents = jet.constituents()
        print("{0:5d} {1:10.3f} {2:10.4f} {3:10.4f} {4:10.3f}".format(
            ijet, jet.pt(), jet.rap(), jet.phi(), len(constituents)))

In [6]:
pythia = pythia8.Pythia()
pythia.readString("Beams:eCM = 8000.")
pythia.readString("HardQCD:all = on")
pythia.readString("PhaseSpace:pTHatMin = 20.")
pythia.readString("Next:numberShowEvent = 0")
pythia.readString("Next:numberShowInfo = 0")
pythia.readString("Next:numberShowProcess = 0")
pythia.init()

True

In [7]:
# set up our jet definition and a jet selector
jet_R0 = 0.4
jet_def = fj.JetDefinition(fj.antikt_algorithm, jet_R0)
selector = fj.SelectorPtMin(20.0) & fj.SelectorAbsRapMax(1)
sd = rt.SoftDrop(0, 0.1, 1.0)

In [8]:
all_jets = []
all_sd_jets = []

In [10]:
from tqdm import tnrange, tqdm_notebook

In [ ]:
for iEvent in tqdm_notebook(range(10000), 'event'):
    # print ("iEvent: ", iEvent)
    if not pythia.next(): continue
    parts = pyfj.vectorize(pythia, True, -1, 1)
    # print(" - number of particles: ", len(parts))
    jets = selector(jet_def(parts))
    sd_jets = [sd.result(j) for j in jets]
    all_jets.extend(jets)
    all_sd_jets.extend(sd_jets)
    # print_jets(jets)
pythia.stat();

In [11]:
def deltas(jets, jets0):
    for i in range(len(jets)):
        yield jets0[i].perp() - jets[i].perp()

In [19]:
etas = [j.eta() for j in all_jets]
pts = [j.pt() for j in all_jets]
sd_pts = [j.pt() for j in all_sd_jets]
sd_delta_pt = [delta for delta in deltas(all_jets, all_sd_jets)]

angs0 = [pyfj.angularity(j, 0.) for j in all_jets]
sd_angs0 = [pyfj.angularity(j, 0.) for j in all_sd_jets]
angs0_R0 = [pyfj.angularity(j, 0., jet_R0) for j in all_jets]
sd_angs0_R0 = [pyfj.angularity(j, 0., jet_R0) for j in all_sd_jets]

angs1 = [pyfj.angularity(j, 1.) for j in all_jets]
sd_angs1 = [pyfj.angularity(j, 1.) for j in all_sd_jets]
angs1_R0 = [pyfj.angularity(j, 1., jet_R0) for j in all_jets]
sd_angs1_R0 = [pyfj.angularity(j, 1., jet_R0) for j in all_sd_jets]

angs15 = [pyfj.angularity(j, 1.5) for j in all_jets]
sd_angs15 = [pyfj.angularity(j, 1.5) for j in all_sd_jets]
angs15_R0 = [pyfj.angularity(j, 1.5, jet_R0) for j in all_jets]
sd_angs15_R0 = [pyfj.angularity(j, 1.5, jet_R0) for j in all_sd_jets]

In [20]:
# %matplotlib notebook
# %matplotlib inline
%matplotlib widget

import matplotlib.pyplot as plt

In [25]:
fig, axes = plt.subplots(nrows=1, ncols=2, sharex=False, sharey=False)
ax0, ax1, = axes.flatten()
n, bins, patches = ax0.hist(pts, 50, density=1, facecolor='blue', alpha=0.3, label='anti-$k_{T}$ R=0.4')
n, bins, patches = ax0.hist(sd_pts, 50, density=1, facecolor='red', alpha=0.3, label='Soft Dropped (SD)')
# n, bins, patches = ax0.hist(sd_pts, 50, density=1, facecolor='red', alpha=0.3)
ax0.set_xlabel(r'$p_{T}$ (GeV)')
ax0.set_ylabel('Probability within $\hat{p_{T}} > 20$')
ax0.set_title(r'$\mathrm{PYTHIA\ jets}\ \sqrt{s}=8\ \mathrm{TeV}$ proton-proton')
ax0.grid(True)
ax0.legend(prop={'size': 10})
ax0.set_yscale('log')

n, bins, patches = ax1.hist(sd_delta_pt, 50, density=1, facecolor='green', alpha=0.3, label='$\Delta p_{T} = p_{T}^{SD} - p_{T}$')
ax1.legend(prop={'size': 10})
ax1.grid(True)
ax1.set_yscale('log')
fig.tight_layout()

FigureCanvasNbAgg()

In [24]:
fig1, axes1 = plt.subplots(nrows=3, ncols=2, sharex=False, sharey=False)
ax10, ax11, ax12, ax13, ax14, ax15= axes1.flatten()

ax10.set_title(r'angularity $\alpha = 0$')
n, bins, patches = ax10.hist(angs0, 25, density=1, facecolor='blue', alpha=0.3)
n, bins, patches = ax10.hist(sd_angs0, 25, density=1, facecolor='red', alpha=0.3)

ax11.set_title(r'scaled by $R_{0}$')
n, bins, patches = ax11.hist(angs0_R0, 25, density=1, facecolor='blue', alpha=0.3)
n, bins, patches = ax11.hist(sd_angs0_R0, 25, density=1, facecolor='red', alpha=0.3)

ax12.set_title(r'angularity $\alpha = 1$')
n, bins, patches = ax12.hist(angs1, 25, density=1, facecolor='blue', alpha=0.3)
n, bins, patches = ax12.hist(sd_angs1, 25, density=1, facecolor='red', alpha=0.3)

ax13.set_title(r'scaled by $R_{0}$')
n, bins, patches = ax13.hist(angs1_R0, 25, density=1, facecolor='blue', alpha=0.3)
n, bins, patches = ax13.hist(sd_angs1_R0, 25, density=1, facecolor='red', alpha=0.3)

ax14.set_title(r'angularity $\alpha = 1.5$')
n, bins, patches = ax14.hist(angs15, 25, density=1, facecolor='blue', alpha=0.3)
n, bins, patches = ax14.hist(sd_angs15, 25, density=1, facecolor='red', alpha=0.3)

ax15.set_title(r'scaled by $R_{0}$')
n, bins, patches = ax15.hist(angs15_R0, 25, density=1, facecolor='blue', alpha=0.3)
n, bins, patches = ax15.hist(sd_angs15_R0, 25, density=1, facecolor='red', alpha=0.3)

fig1.tight_layout()


FigureCanvasNbAgg()